# Vertex Tabular Binary Classification with Custom Jobs

<center><img src="../images/03.png"/></center>

## Set Constants

In [2]:
PROJECT_ID = 'jchavezar-demo'
REGION = 'us-central1'
DIR = '/tmp'
DATASET_URI = 'gs://vtx-datasets-public/ecommerce/train.csv'
MODEL_URI = 'gs://vtx-models/ecommerce'
STAGING_URI = 'gs://vtx-staging/ecommerce/'
TRAIN_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-9:latest'
PREDICTION_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-9:latest'

## Create Folder Structure

```
tmp
└─── source
     └─── trainer
          |  train.py
          |

```

In [4]:
!rm -fr {DIR}/source
!mkdir -p {DIR}/source/trainer

In [21]:
%%writefile {DIR}/source/trainer/train.py


import os
import pandas as pd
from google.cloud import storage

bucket = os.environ['AIP_TRAINING_DATA_URI'].split('/')[2]
train_prefix = '/'.join(os.environ['AIP_TRAINING_DATA_URI'].split('/')[3:])[:-1]
val_prefix = '/'.join(os.environ['AIP_VALIDATION_DATA_URI'].split('/')[3:])[:-1]
test_prefix = '/'.join(os.environ['AIP_TEST_DATA_URI'].split('/')[3:])[:-1]

df = pd.DataFrame()
client = storage.Client()
bucket = client.bucket(bucket)

train_df_list = []

def create_dataset(train_prefix: str):
    df_list = []
    for file in list(bucket.list_blobs(prefix=train_prefix)):
        file_path = f"gs://{file.bucket.name}/{file.name}"
        df_list.append(pd.read_csv(file_path))
        return pd.concat(df_list)

train_df = create_dataset(train_prefix)
val_df = create_dataset(val_prefix)
test_df = create_dataset(test_prefix)

print(train_df.shape)
print(val_df.shape)
print(test_df.shape)
print(train_df)

Overwriting /tmp/source/trainer/train.py


In [22]:
%%writefile {DIR}/source/trainer/train.py

import os
import time
import warnings
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import storage
from tensorflow.keras import layers
warnings.filterwarnings('ignore')

################################### DATASETS #######################################


bucket = os.environ['AIP_TRAINING_DATA_URI'].split('/')[2]
train_prefix = '/'.join(os.environ['AIP_TRAINING_DATA_URI'].split('/')[3:])[:-1]
val_prefix = '/'.join(os.environ['AIP_VALIDATION_DATA_URI'].split('/')[3:])[:-1]
test_prefix = '/'.join(os.environ['AIP_TEST_DATA_URI'].split('/')[3:])[:-1]

df = pd.DataFrame()
client = storage.Client()
bucket = client.bucket(bucket)

train_df_list = []

def create_dataset(train_prefix: str):
    df_list = []
    for file in list(bucket.list_blobs(prefix=train_prefix)):
        file_path = f"gs://{file.bucket.name}/{file.name}"
        df_list.append(pd.read_csv(file_path))
        return pd.concat(df_list)

train_df = create_dataset(train_prefix)
val_df = create_dataset(val_prefix)
test_df = create_dataset(test_prefix)


################################### PREPROCESSING #######################################

## Convert pandas dataframe to tensor data (from GCS to TF.data.Data)
init_start = time.process_time()
def df_to_dataset(dataframe, shuffle=None):
    df = dataframe.copy()
    labels = df.pop('will_buy_on_return_visit')
    df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds
      
## Normalization / Standarization
def get_normalization_layer(name, dataset):
    start = time.process_time()
    normalizer = layers.Normalization(axis=None)
    feature_ds = dataset.map(lambda x, y: x[name])
    normalizer.adapt(feature_ds)
    print(f'Normalization time for {name}: {time.process_time() - start}')
    return normalizer

# Performs feature-wise categorical encoding of inputs features
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    start = time.process_time()
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
    feature_ds = dataset.map(lambda x, y: x[name])
    index.adapt(feature_ds)
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    print(f'Encoding time for {name}: {time.process_time() - start}')
    return lambda feature: encoder(index(feature))

batch_size = 256
train_ds = df_to_dataset(train_df)
val_ds = df_to_dataset(val_df)
test_ds = df_to_dataset(test_df)

## Identify Numerical and Categorical columns:
num_columns = ['latest_ecommerce_progress', 'time_on_site', 'pageviews']
cat_columns = ['source', 'medium', 'channelGrouping', 'deviceCategory', 'country']
num_cat_columns = 'bounces'

all_inputs = []
encoded_features = []

# Numerical Features.
for header in num_columns:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)
    
# Categorical Features.
for header in cat_columns:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(name=header,
                                                 dataset=train_ds,
                                                 dtype='string',
                                                 max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

## Integer values into integer indices.
bounces_col = tf.keras.Input(shape=(1,), name=num_cat_columns, dtype='int64')

encoding_layer = get_category_encoding_layer(name=num_cat_columns,
                                             dataset=train_ds,
                                             dtype='int64',
                                             max_tokens=5)
encoded_age_col = encoding_layer(bounces_col)
all_inputs.append(bounces_col)
encoded_features.append(encoded_age_col)

print(f'Total preprocessing time: {time.process_time() - init_start}')

#########################################################################################


################################### CREATE, COMPILE AND TRAIN MODEL #####################

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)


model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

model.fit(train_ds, epochs=10, validation_data=val_ds)
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

################################### SAVE MODEL ##########################################

model.save(os.environ['AIP_MODEL_DIR'])

Overwriting /tmp/source/trainer/train.py


## Create Vertex Managed Dataset

In [19]:
from google.cloud import aiplatform as aip

dataset = aip.TabularDataset.create(
    display_name = 'ecommerce_tabular',
    gcs_source = DATASET_URI,
    
)

Creating TabularDataset
Create TabularDataset backing LRO: projects/569083142710/locations/us-central1/datasets/4245260574331502592/operations/3754701710568718336
TabularDataset created. Resource name: projects/569083142710/locations/us-central1/datasets/4245260574331502592
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/569083142710/locations/us-central1/datasets/4245260574331502592')


In [25]:
pipeline_job = aip.CustomTrainingJob(
    display_name = 'tf_tab_pipe_training_ecommerce',
    script_path = f'{DIR}/source/trainer/train.py',
    container_uri = TRAIN_IMAGE_URI,
    staging_bucket = STAGING_URI,
    model_serving_container_image_uri = PREDICTION_IMAGE_URI,
)

model = pipeline_job.run(
    dataset = dataset,
    base_output_dir = MODEL_URI+'/model',
    model_display_name = 'tf_tab_pipe_training_ecommerce',
    training_fraction_split = 0.8,
    validation_fraction_split = 0.1,
    test_fraction_split = 0.1
)

Training script copied to:
gs://vtx-staging/ecommerce/aiplatform-2022-10-24-18:46:44.470-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://vtx-models/ecommerce/model 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8540318854918701056?project=569083142710
CustomTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/8540318854918701056 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/8540318854918701056 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/8540318854918701056 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/8540318854918701056 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/569083142710/locations/us-central1/trai

## Get Model Information from Tensorflow Graph

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

loaded_model = tf.keras.models.load_model(f"{MODEL_URI}/model")
tf.keras.utils.plot_model(loaded_model, show_shapes=True, rankdir="LR")

## Deploy Model On Endpoint

In [ ]:
endpoint = model.deploy(
    deployed_model_display_name = 'ecommerce_tf_ep_dep',
    traffic_percentage = 100,
    machine_type = 'n1-standard-4',
    min_replica_count = 1,
    max_replica_count = 1,
    #explanation_metadata=EXPLANATION_METADATA,
    #explanation_parameters=EXPLANATION_PARAMS
)

## Testing Predictions

In [ ]:
instance = {
    'latest_ecommerce_progress': [0],
    'bounces': [0],
    'time_on_site': [103],
    'pageviews': [3],
    'source': ['youtube.com'],
    'medium': ['referral'],
    'channelGrouping': ['Social'],
    'deviceCategory': ['desktop'],
    'country': ['Vietnam'],
}

In [ ]:
endpoint.predict([instance])

In [ ]:
explanations = endpoint.explain([instance])
print("Explainable predictions:", explanations)